In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from scipy.sparse import coo_matrix
import pickle
import numpy as np
import scipy.sparse as sp

In [2]:
train_path = "/public/home/zhemodedc/ReChorus-master/data/MovieLens_1M/train.csv"
train_data = pd.read_csv(train_path, sep='\t')

train_data.sample(5)

,user_id,item_id,time,c_hour_c,c_weekday_c,c_period_c,c_day_f
329349,2827,865,966868705.0,22,0,7,117
10835,524,844,977357720.0,8,3,0,239
233970,1677,1344,974082528.0,10,0,1,201
459934,4310,319,963046374.0,16,5,4,73
312398,2658,607,967657478.0,1,3,8,127


In [3]:
test_path = "/public/home/zhemodedc/ReChorus-master/data/MovieLens_1M/test.csv"
test_data = pd.read_csv(test_path, sep='\t')

test_data.sample(5)

,user_id,item_id,time,c_hour_c,c_weekday_c,c_period_c,c_day_f,neg_items
272,434,243,1.038286e+09,12,1,2,944,"[782, 1941, 1366, 1292, 2964, 475, 1191, 1591,..."
2081,2238,2219,1.044067e+09,10,5,1,1011,"[2451, 2038, 2672, 125, 2385, 2806, 416, 1243,..."
2340,4392,584,1.044647e+09,3,5,8,1018,"[1610, 2506, 2505, 70, 209, 393, 3005, 536, 18..."
650,1435,2180,1.039502e+09,14,1,3,958,"[605, 1692, 1170, 2610, 2671, 677, 2192, 3125,..."
1806,1053,2016,1.043212e+09,13,2,3,1001,"[1230, 2996, 802, 2106, 2043, 1611, 404, 1793,..."


In [4]:
val_path = "/public/home/zhemodedc/ReChorus-master/data/MovieLens_1M/dev.csv"
val_data = pd.read_csv(val_path, sep='\t')

val_data.sample(5)

,user_id,item_id,time,c_hour_c,c_weekday_c,c_period_c,c_day_f,neg_items
620,2206,2578,1.030053e+09,5,4,0,849,"[490, 467, 2372, 2577, 442, 1113, 1289, 764, 2..."
1799,100,1738,1.035406e+09,4,3,8,911,"[2473, 1666, 228, 1391, 1402, 2251, 2920, 2336..."
449,4392,1070,1.029941e+09,22,2,7,847,"[882, 1784, 401, 2972, 3118, 810, 2376, 864, 2..."
2344,841,265,1.036918e+09,16,6,4,928,"[536, 2929, 463, 70, 1080, 1964, 3089, 1782, 2..."
642,499,2271,1.030087e+09,15,4,3,849,"[180, 2707, 1065, 2465, 1699, 1724, 2245, 1362..."


In [5]:
def create_sparse_matrix(data):
    # Extract relevant columns (user_id, item_id, time)
    data = data[['user_id', 'item_id', 'time']]

    # Convert user_id and item_id to zero-indexed
    user_id_map = {user_id: idx for idx, user_id in enumerate(data['user_id'].unique())}
    item_id_map = {item_id: idx for idx, item_id in enumerate(data['item_id'].unique())}

    data['user_id'] = data['user_id'].map(user_id_map)
    data['item_id'] = data['item_id'].map(item_id_map)

    # Create sparse matrix using COO format (Row, Column, Value)
    row = data['user_id'].values
    col = data['item_id'].values
    interaction_data = np.ones(len(data))  # Interaction strength (binary 1 for interaction)

    interaction_matrix = sp.coo_matrix((interaction_data, (row, col)), shape=(len(user_id_map), len(item_id_map)))

    return interaction_matrix, user_id_map, item_id_map

In [6]:
train_matrix, train_user_map, train_item_map = create_sparse_matrix(train_data)
test_matrix, test_user_map, test_item_map = create_sparse_matrix(test_data)
val_matrix, val_user_map, val_item_map = create_sparse_matrix(val_data)

/tmp/ipykernel_2669056/2631126017.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['user_id'] = data['user_id'].map(user_id_map)
/tmp/ipykernel_2669056/2631126017.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['item_id'] = data['item_id'].map(item_id_map)
/tmp/ipykernel_2669056/2631126017.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pa

In [7]:
train_output_path = '/public/home/zhemodedc/tianchi/GFormer/Datasets/movie_lens/trnMat.pkl'
test_output_path = '/public/home/zhemodedc/tianchi/GFormer/Datasets/movie_lens/tstMat.pkl'
val_output_path = '/public/home/zhemodedc/tianchi/GFormer/Datasets/movie_lens/valMat.pkl'

train_user_map_path = '/public/home/zhemodedc/tianchi/GFormer/Datasets/movie_lens/train_user_map.csv'
train_item_map_path = '/public/home/zhemodedc/tianchi/GFormer/Datasets/movie_lens/train_item_map.csv'

test_user_map_path = '/public/home/zhemodedc/tianchi/GFormer/Datasets/movie_lens/test_user_map.csv'
test_item_map_path = '/public/home/zhemodedc/tianchi/GFormer/Datasets/movie_lens/test_item_map.csv'

val_user_map_path = '/public/home/zhemodedc/tianchi/GFormer/Datasets/movie_lens/val_user_map.csv'
val_item_map_path = '/public/home/zhemodedc/tianchi/GFormer/Datasets/movie_lens/val_item_map.csv'

In [8]:
train_user_map_df = pd.DataFrame(list(train_user_map.items()), columns=["real_user_id", "zero_indexed_user_id"])
train_item_map_df = pd.DataFrame(list(train_item_map.items()), columns=["real_item_id", "zero_indexed_item_id"])

test_user_map_df = pd.DataFrame(list(test_user_map.items()), columns=["real_user_id", "zero_indexed_user_id"])
test_item_map_df = pd.DataFrame(list(test_item_map.items()), columns=["real_item_id", "zero_indexed_item_id"])

val_user_map_df = pd.DataFrame(list(val_user_map.items()), columns=["real_user_id", "zero_indexed_user_id"])
val_item_map_df = pd.DataFrame(list(val_item_map.items()), columns=["real_item_id", "zero_indexed_item_id"])

In [9]:
train_user_map_df.to_csv(train_user_map_path, index=False)
train_item_map_df.to_csv(train_item_map_path, index=False)

test_user_map_df.to_csv(test_user_map_path, index=False)
test_item_map_df.to_csv(test_item_map_path, index=False)

val_user_map_df.to_csv(val_user_map_path, index=False)
val_item_map_df.to_csv(val_item_map_path, index=False)

In [10]:
with open(train_output_path, 'wb') as file:
    pickle.dump(train_matrix, file)

with open(test_output_path, 'wb') as file:
    pickle.dump(test_matrix, file)

with open(val_output_path, 'wb') as file:
    pickle.dump(val_matrix, file)

In [51]:
def verify_pkl(file_path):
    with open(file_path, 'rb') as file:
        matrix_data = pickle.load(file)

    # Print details about the loaded matrix
    print(f"Matrix loaded from {file_path}:")
    print("Shape of the matrix:", matrix_data.shape)
    print("Number of non-zero elements:", matrix_data.nnz)
    print("Row indices (users):", matrix_data.row)
    print("Column indices (items):", matrix_data.col)
    print("Values (interaction strength):", matrix_data.data)
    print("\n")

In [52]:
verify_pkl(train_output_path)
verify_pkl(test_output_path)
verify_pkl(val_output_path)

Matrix loaded from /public/home/zhemodedc/tianchi/GFormer/Datasets/movie_lens/trnMat.pkl:
Shape of the matrix: (6032, 3125)
Number of non-zero elements: 568761
Row indices (users): [   0    0    0 ... 6031 6031 6031]
Column indices (items): [  0   1   2 ... 212 100  19]
Values (interaction strength): [1. 1. 1. ... 1. 1. 1.]


Matrix loaded from /public/home/zhemodedc/tianchi/GFormer/Datasets/movie_lens/tstMat.pkl:
Shape of the matrix: (226, 1260)
Number of non-zero elements: 2874
Row indices (users): [  0   0   0 ... 205 205 205]
Column indices (items): [   0    1    2 ... 1258 1054 1259]
Values (interaction strength): [1. 1. 1. ... 1. 1. 1.]


Matrix loaded from /public/home/zhemodedc/tianchi/GFormer/Datasets/movie_lens/valMat.pkl:
Shape of the matrix: (214, 1198)
Number of non-zero elements: 2562
Row indices (users): [ 0  1  1 ... 27 37 79]
Column indices (items): [  0   1   2 ... 634  25 846]
Values (interaction strength): [1. 1. 1. ... 1. 1. 1.]




In [31]:
train_user_map = pd.read_csv(train_user_map_path).to_dict()

In [38]:
train_user_map.keys()

dict_keys(['real_user_id', 'zero_indexed_user_id'])

In [39]:
train_user_map.values()

dict_values([{0: 1, 1: 1112, 2: 2222, 3: 3332, 4: 4438, 5: 5548, 6: 5700, 7: 5811, 8: 5922, 9: 2, 10: 113, 11: 224, 12: 335, 13: 446, 14: 557, 15: 668, 16: 779, 17: 890, 18: 1001, 19: 1113, 20: 1224, 21: 1335, 22: 1446, 23: 1557, 24: 1668, 25: 1779, 26: 1890, 27: 2001, 28: 2112, 29: 2223, 30: 2334, 31: 2445, 32: 2556, 33: 2667, 34: 2778, 35: 2888, 36: 2999, 37: 3110, 38: 3221, 39: 3333, 40: 3444, 41: 3555, 42: 3665, 43: 3775, 44: 3884, 45: 3995, 46: 4105, 47: 4216, 48: 4327, 49: 4439, 50: 4550, 51: 4661, 52: 4772, 53: 4883, 54: 4994, 55: 5105, 56: 5216, 57: 5327, 58: 5437, 59: 5549, 60: 5601, 61: 5612, 62: 5623, 63: 5634, 64: 5645, 65: 5656, 66: 5667, 67: 5678, 68: 5689, 69: 5701, 70: 5712, 71: 5723, 72: 5734, 73: 5745, 74: 5756, 75: 5767, 76: 5778, 77: 5789, 78: 5800, 79: 5812, 80: 5823, 81: 5834, 82: 5845, 83: 5856, 84: 5867, 85: 5878, 86: 5889, 87: 5900, 88: 5911, 89: 5923, 90: 5934, 91: 5945, 92: 5956, 93: 5967, 94: 5978, 95: 5989, 96: 6000, 97: 6011, 98: 6022, 99: 3, 100: 14, 101:

In [47]:
train_user_map['real_user_id'][2]

2222

In [42]:
train_user_map['real_user_id']

{0: 1,
 1: 1112,
 2: 2222,
 3: 3332,
 4: 4438,
 5: 5548,
 6: 5700,
 7: 5811,
 8: 5922,
 9: 2,
 10: 113,
 11: 224,
 12: 335,
 13: 446,
 14: 557,
 15: 668,
 16: 779,
 17: 890,
 18: 1001,
 19: 1113,
 20: 1224,
 21: 1335,
 22: 1446,
 23: 1557,
 24: 1668,
 25: 1779,
 26: 1890,
 27: 2001,
 28: 2112,
 29: 2223,
 30: 2334,
 31: 2445,
 32: 2556,
 33: 2667,
 34: 2778,
 35: 2888,
 36: 2999,
 37: 3110,
 38: 3221,
 39: 3333,
 40: 3444,
 41: 3555,
 42: 3665,
 43: 3775,
 44: 3884,
 45: 3995,
 46: 4105,
 47: 4216,
 48: 4327,
 49: 4439,
 50: 4550,
 51: 4661,
 52: 4772,
 53: 4883,
 54: 4994,
 55: 5105,
 56: 5216,
 57: 5327,
 58: 5437,
 59: 5549,
 60: 5601,
 61: 5612,
 62: 5623,
 63: 5634,
 64: 5645,
 65: 5656,
 66: 5667,
 67: 5678,
 68: 5689,
 69: 5701,
 70: 5712,
 71: 5723,
 72: 5734,
 73: 5745,
 74: 5756,
 75: 5767,
 76: 5778,
 77: 5789,
 78: 5800,
 79: 5812,
 80: 5823,
 81: 5834,
 82: 5845,
 83: 5856,
 84: 5867,
 85: 5878,
 86: 5889,
 87: 5900,
 88: 5911,
 89: 5923,
 90: 5934,
 91: 5945,
 92: 5956,
 

In [32]:
real_user_ids = train_user_map['real_user_id']
zero_indexed_user_ids = train_user_map['zero_indexed_user_id']

In [33]:
reversed_train_user_map = dict(zip(zero_indexed_user_ids, real_user_ids))

In [34]:
reversed_train_user_map.keys()

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219,

In [35]:
reversed_train_user_map.values()

dict_values([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 21

In [48]:
train_item_map = pd.read_csv(train_item_map_path).to_dict()
train_item_map = train_item_map['real_item_id']